In [3]:
import pandas as pd

df = pd.read_csv("audience.csv")

mau = df['user_id'].nunique()
print("MAU (уникальных пользователей в ноябре):", mau)


MAU (уникальных пользователей в ноябре): 7639


In [4]:
dau_per_day = df.groupby('date')['user_id'].nunique()

print(dau_per_day)

average_dau = dau_per_day.mean()
print("Средний DAU за ноябрь:", average_dau)

date
2023-11-01    623
2023-11-02    649
2023-11-03    573
2023-11-04    343
2023-11-05    350
2023-11-06    660
2023-11-07    629
2023-11-08    600
2023-11-09    661
2023-11-10    583
2023-11-11    354
2023-11-12    377
2023-11-13    646
2023-11-14    687
2023-11-15    690
2023-11-16    639
2023-11-17    585
2023-11-18    412
2023-11-19    378
2023-11-20    711
2023-11-21    651
2023-11-22    608
2023-11-23    632
2023-11-24    595
2023-11-25    366
2023-11-26    372
2023-11-27    589
2023-11-28    610
2023-11-29    621
2023-11-30    620
Name: user_id, dtype: int64
Средний DAU за ноябрь: 560.4666666666667


In [7]:
print(df.columns)


Index(['date', 'user_id', 'view_adverts'], dtype='object')


In [8]:
df['date'] = pd.to_datetime(df['date'])

new_users = df[df['date'] == '2023-11-01']['user_id'].unique()

returned_users = df[(df['date'] == '2023-11-02') & (df['user_id'].isin(new_users))]['user_id'].nunique()

# Retention
retention = returned_users / len(new_users) * 100
print(f"Day 1 Retention: {retention:.1f}%")


Day 1 Retention: 26.6%


In [9]:
total_users = df['user_id'].nunique()

users_with_views = df[df['view_adverts'] > 0]['user_id'].nunique()

conversion = (users_with_views / total_users) * 100

print(f'Конверсия в просмотр объявления: {conversion:.2f}%')

Конверсия в просмотр объявления: 46.31%


In [12]:
active_users_df = df[df['view_adverts'] > 0]

active_users_count = active_users_df['user_id'].nunique()

total_views = active_users_df['view_adverts'].sum()

average_views = total_views / active_users_count

print(f'Среднее количество просмотров на активного пользователя: {average_views:.1f}')

Среднее количество просмотров на активного пользователя: 6.2


In [13]:
total_users = 2000
promoters = 1200
passives = 300
detractors = 500

promoters_percent = (promoters / total_users) * 100
detractors_percent = (detractors / total_users) * 100

# NPS = % Promoters - % Detractors
nps = promoters_percent - detractors_percent

print(f'NPS: {nps:.0f}%')


NPS: 35%


In [17]:
import pandas as pd

df_ab_test = pd.read_excel('audience.xlsx', sheet_name=1)

print(df_ab_test.head())


   experiment_num experiment_group   user_id  revenue
0               1             test     38456      520
1               1          control  13125924      806
2               1          control   9761984        0
3               1             test  11387012      208
4               1             test  18319648      104


In [18]:
from scipy.stats import ttest_ind

for exp in df_ab_test['experiment_num'].unique():
    exp_data = df_ab_test[df_ab_test['experiment_num'] == exp]
    control = exp_data[exp_data['experiment_group'] == 'control']['revenue']
    test = exp_data[exp_data['experiment_group'] == 'test']['revenue']
    
    arpu_control = control.mean()
    arpu_test = test.mean()

    stat, p_value = ttest_ind(test, control, equal_var=False)

    print(f'Эксперимент {exp}:')
    print(f'ARPU Control: {arpu_control:.2f}')
    print(f'ARPU Test: {arpu_test:.2f}')
    print(f'p-value: {p_value:.4f}\n')


Эксперимент 1:
ARPU Control: 722.46
ARPU Test: 665.74
p-value: 0.6890

Эксперимент 2:
ARPU Control: 704.65
ARPU Test: 332.93
p-value: 0.0011

Эксперимент 3:
ARPU Control: 663.21
ARPU Test: 998.67
p-value: 0.0603



In [19]:
import pandas as pd

df_listers = pd.read_excel('audience.xlsx', sheet_name=2)

total_revenue = df_listers['revenue'].sum()

total_users = df_listers['user_id'].nunique()

arpu = total_revenue / total_users

print(f'Средний доход на пользователя (ARPU): {arpu:.1f}')

Средний доход на пользователя (ARPU): 156.5


In [20]:
median_age = df_listers['age'].median()

print(f'Медиана возраста: {median_age:.2f}')

Медиана возраста: 28.00


In [21]:
import statsmodels.api as sm

visitors_A = 100047501
payments_A = 1003

visitors_B = 100001055
payments_B = 1099

successes = [payments_A, payments_B]
observations = [visitors_A, visitors_B]

z_stat, p_value = sm.stats.proportions_ztest(successes, observations)

print(f'Z-статистика: {z_stat:.4f}')
print(f'p-value: {p_value:.4f}')

if p_value < 0.05:
    print('Результат статистически значим. Можно рекомендовать внедрение изменений.')
else:
    print('Различие не является статистически значимым. Внедрять изменения не рекомендуется.')


Z-статистика: -2.1046
p-value: 0.0353
Результат статистически значим. Можно рекомендовать внедрение изменений.
